# **Section0502_Group04_Project_B**

### Our choices

In [1]:
# For grading.
maFast = 11
maSlow = 220
bbWindow = 23
bbStdevBand = 2
myStrategies = [
    'AAPL-MAFlat', 'AMZN-MAFlat', 'EUR-MAShort',
    'FBNDX-MAShort', 'GE-MAShort', 'GOLD-MAFlat',
    'INTC-BB', 'SPY-MAFlat'
]

### Importing Required Libraraies:

In [2]:
# import necessary libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

### Defining all the required functions:

In [3]:
def read_price_data(filename, index_col=0, parse_dates=True):
    # Reading the CSV file
    df = pd.read_csv(filename, index_col=index_col, parse_dates=parse_dates)

    # Printing dataframe information
    print("Dataframe info:")
    print(df.info())

    # Checking for any null values
    print("\nChecking for any null values:")
    print(df.isnull().sum())

    # Printing summary statistics
    print("\nSummary statistics:")
    print(df.describe())

    # Displaying the dataframe
    print("\nDisplaying dataFrame:")
    print(df)
    return df

In [4]:
def print_pct_data(data_frame, stringFmtSpec='{:.4%}', na_rep='NaN', index_slice=None):
    if index_slice is not None:
        styler = data_frame.style.format(na_rep=na_rep)
        styler = styler.format(stringFmtSpec, subset=pd.IndexSlice[index_slice, :])
    else:
        styler = data_frame.style.format(stringFmtSpec, na_rep=na_rep)

    return styler

In [5]:
def calcPerformanceStatistics(returns, annualization_factor=260, print_Screen=True, percent_format_string='{:.2%}'):
    if pd.isna(returns.iloc[0, 0]):  # Check if the first value is NaN
        if not returns.iloc[0].isna().all():  # Check if all values in the first row are NaN
            raise ValueError("First row should be all NaN or not NaN at all.")
    else:
        returns = pd.concat([pd.DataFrame(index=[returns.index[0] - pd.Timedelta(days=1)], columns=returns.columns), returns])

    # assign the cumulative returns to a new variable which will have the first NaN row
    cumReturns = returns.cumsum()
    # verify replace the NaN row with zeroes
    cumReturns = cumReturns.fillna(0)
    # Calculate High Water Mark (HWM)
    high_water_mark = np.maximum.accumulate(cumReturns)
    underWaterCurve = cumReturns - high_water_mark

    average_annual_return = annualization_factor * returns.mean()
    annual_risk = np.sqrt(annualization_factor) * returns.std()
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(annualization_factor)
    max_drawdown = underWaterCurve.min()
    success_ratio = (returns.dropna() >= 0).sum() / returns.dropna().shape[0]


    summary_stats = pd.DataFrame([
        average_annual_return,
        annual_risk,
        sharpe_ratio,
        max_drawdown,
        success_ratio
    ], index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'Max Drawdown', 'Success Ratio'])

    if print_Screen:
        print("Performance Statistics:")
        print_pct_data(summary_stats, percent_format_string)

    return summary_stats

In [6]:
def movingAverageCrossover(prices, fastWindow, slowWindow):
    prices = prices.copy()
    if prices.shape[1] > 1:
        raise ValueError("Input dataFrame should have only one column.")

    # Calculate moving averages
    prices.loc[:,"FastSMA"] = prices[f"{prices.columns[0]}"].rolling(window=fastWindow).mean()
    prices.loc[:,"SlowSMA"] = prices[f"{prices.columns[0]}"].rolling(window=slowWindow).mean()
    prices.dropna(inplace=True)

    # Benchmark (Buy and Hold)
    prices.loc[:,f"{prices.columns[0]}-BMK-MA"] = np.log(prices[f"{prices.columns[0]}"] / prices[f"{prices.columns[0]}"].shift(1))

    # "Go Flat" strategy (flat when fast MA is below slow MA)
    prices.loc[:,'PositionFlat'] = np.where(prices['FastSMA'] > prices['SlowSMA'], 1, 0)  # 1 if fast > slow, 0 otherwise
    prices.style.format(na_rep='NaN')
    flat_returns = prices['PositionFlat'].shift(1) * prices[f"{prices.columns[0]}-BMK-MA"]

    # "Go Short" strategy (short when fast MA is below slow MA)
    prices.loc[:,'PositionShort'] = np.where(prices['FastSMA'] > prices['SlowSMA'], 1, -1)  # 1 for long, -1 for short
    prices.style.format(na_rep='NaN')
    short_returns = prices['PositionShort'].shift(1) * prices[f"{prices.columns[0]}-BMK-MA"]

    result = pd.DataFrame({
        f"{prices.columns[0]}-BMK-MA": prices[f"{prices.columns[0]}-BMK-MA"].squeeze(),
        f"{prices.columns[0]}-MAFlat": flat_returns.squeeze(),
        f"{prices.columns[0]}-MAShort": short_returns.squeeze()
    }, index=prices.index)

    return result


In [7]:
def bollingerBands(prices, lookbackWindow, stdevBand):
    prices = prices.copy()

    if prices.shape[1] > 1:
        raise ValueError("Input dataFrame should have only one column.")

    # Calculate the moving average and standard deviation
    prices['SMA'] = prices[f"{prices.columns[0]}"].rolling(window=lookbackWindow).mean()
    prices['STD'] = prices[f"{prices.columns[0]}"].rolling(window=lookbackWindow).std()
    prices.dropna(inplace=True)

    # Calculate the upper and lower Bollinger Bands
    prices['UpperBand'] = prices['SMA'] + (stdevBand * prices['STD'])
    prices['LowerBand'] = prices['SMA'] - (stdevBand * prices['STD'])

    # Bollinger Band strategy

    # initialize the first row based on the close price vis-a-vis the bands
    # here, we're also initializng a new column called 'Position'
    date0 = prices.index[0]
    if prices.loc[date0, prices.columns[0]] >= prices.loc[date0, 'UpperBand']:
        prices.loc[date0, 'Position'] = -1
    elif prices.loc[date0, prices.columns[0]] <= prices.loc[date0, 'LowerBand']:
        prices.loc[date0, 'Position'] = 1
    else:
        prices.loc[date0, 'Position'] = 0

    # Loop over the entire data set starting from the second row
    for i in range(1, prices.shape[0]):
        today = prices.index[i]
        yesterday = prices.index[i - 1]

        # If today's price is above the upper band, go short (-1)
        if prices.loc[today, prices.columns[0]] >= prices.loc[today, 'UpperBand']:
            prices.loc[today, 'Position'] = -1

        # If today's price is below the lower band, go long (1)
        elif prices.loc[today, prices.columns[0]] <= prices.loc[today, 'LowerBand']:
            prices.loc[today, 'Position'] = 1

        # If yesterday's position was short and today's price is below the mean, flatten (0)
        elif prices.loc[yesterday, 'Position'] == -1 and prices.loc[today, prices.columns[0]] <= prices.loc[today, 'SMA']:
            prices.loc[today, 'Position'] = 0

        # If yesterday's position was long and today's price is above the mean, flatten (0)
        elif prices.loc[yesterday, 'Position'] == 1 and prices.loc[today, prices.columns[0]] >= prices.loc[today, 'SMA']:
            prices.loc[today, 'Position'] = 0

        # Otherwise, maintain yesterday's position
        else:
            prices.loc[today, 'Position'] = prices.loc[yesterday, 'Position']

    # Benchmark using log returns
    prices[f"{prices.columns[0]}-BMK-BB"] = np.log(prices[f"{prices.columns[0]}"] / prices[f"{prices.columns[0]}"].shift(1))

    # Calculate the returns for the Bollinger Band strategy
    prices[f"{prices.columns[0]}-BB"] = prices['Position'].shift(1) * prices[f"{prices.columns[0]}-BMK-BB"]

    # Drop NaN values that were introduced during calculations
    prices.dropna(inplace=True)

    result = pd.DataFrame({
        f"{prices.columns[0]}-BMK-BB": prices[f"{prices.columns[0]}-BMK-BB"],
        f"{prices.columns[0]}-BB": prices[f"{prices.columns[0]}-BB"]
    }, index=prices.index)

    return result


In [8]:
import pandas as pd

def runMovingAverageAndBB(dataFrame, fastWindow, slowWindow, bbWindow, stdevBand):

    # Run the movingAverageCrossover function
    ma_result = movingAverageCrossover(dataFrame.copy(), fastWindow, slowWindow)

    # Run the bollingerBands function
    bb_result = bollingerBands(dataFrame.copy(), bbWindow, stdevBand)

    # Perform an inner join on the index (date)
    result = ma_result.join(bb_result, how='inner').dropna()

    # Ensure that the columns for BMK-MA and BMK-BB are identical
    bmk_ma_col = [col for col in result.columns if 'BMK-MA' in col][0]
    bmk_bb_col = [col for col in result.columns if 'BMK-BB' in col][0]

    if not result[bmk_ma_col].equals(result[bmk_bb_col]):
        raise ValueError("BMK-MA and BMK-BB columns should be identical.")

    # Return the joined DataFrame with 5 columns
    return result


## Q1.  Importing Close Price Data for Multiple Asset Classes (till Oct 2022)

In [9]:
# Importing Prices csv file.
dfPrices = pd.read_csv('PricesProjectB.csv', index_col=0, parse_dates=True)

In [10]:
dfPrices

,AAPL,AMZN,ATT,AUD,EUR,FBNDX,GE,GOLD,INTC,LEHM,SILVER,SPY
1999-12-31,0.917969,3.806250,36.820244,0.6567,1.0062,6.89,396.794861,288.00,41.156250,798.154631,5.3800,146.875000
2000-01-03,0.999442,4.468750,35.498489,0.6585,1.0243,6.85,384.615387,289.00,43.500000,793.868670,5.4050,145.437500
2000-01-04,0.915179,4.096875,33.421452,0.6550,1.0296,6.87,369.230774,282.25,41.468750,796.761677,5.3250,139.750000
2000-01-05,0.928571,3.487500,33.940708,0.6579,1.0321,6.84,368.589752,280.00,41.812500,793.118215,5.1450,140.000000
2000-01-06,0.848214,3.278125,33.043808,0.6533,1.0328,6.86,373.517639,281.10,39.375000,795.091984,5.1200,137.750000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,152.339996,120.599998,17.690001,0.6394,0.9966,6.88,73.000000,1653.17,27.410000,844.938544,19.3473,384.920013
2022-10-26,149.350006,115.660004,18.139999,0.6497,1.0081,6.91,75.459999,1664.57,27.209999,849.013953,19.5815,382.019989
2022-10-27,144.800003,110.959999,18.030001,0.6452,0.9964,6.94,76.000000,1663.31,26.270000,853.391436,19.6014,379.980011
2022-10-28,155.740005,103.410004,18.480000,0.6411,0.9965,6.93,78.330002,1644.86,29.070000,851.238239,19.2595,389.019989


## Q3. Re-run strategies on complete window.

In [11]:
# List of all 12 instruments
instruments = ['AAPL', 'AMZN', 'GE', 'INTC', 'SPY',  # Chosen Equities
               'FBNDX',  # Chosen Fixed Income
               'GOLD',   # Chosen Commodities
               'EUR']    # Chosen Currencies

# Define the parameters based on your chosen values
fastWindow = 11
slowWindow = 220
bbWindow = 23
stdevBand = 2

# Initialize an empty list to store the DataFrames for each instrument
dfs = []

# Loop through each instrument and apply the strategies
for instrument in instruments:

    # Select only the instrument's price data
    data = dfPrices[[instrument]].dropna()

    # Run the combined Moving Average and Bollinger Band strategy
    result = runMovingAverageAndBB(data, fastWindow, slowWindow, bbWindow, stdevBand)

    # Rename the columns to include the instrument name for uniqueness in the master DataFrame
    # result = result.add_prefix(f"{instrument}-")

    # Append the result to the list of DataFrames
    dfs.append(result)

# Concatenate all the individual DataFrames along the columns (axis=1)
master_df = pd.concat(dfs, axis=1)


In [12]:
# Define the 8 chosen strategies in alphabetical order
myStrategies = [
    'AAPL-MAFlat', 'AMZN-MAFlat', 'EUR-MAShort',
    'FBNDX-MAShort', 'GE-MAShort', 'GOLD-MAFlat',
    'INTC-BB', 'SPY-MAFlat'
]

# Create the 'myStrategies' DataFrame by subsetting the master DataFrame
myStrategies_df = master_df[[f"{strategy}" for strategy in myStrategies]].copy()

inSample = pd.IndexSlice[:'2018-12-31',:]
outOfSample = pd.IndexSlice['2019-01-02':'2022-10-31',:]

# Display the first few rows to confirm the subset
print(myStrategies_df.head())

# Verify the shape of the new DataFrame
print(f"My Strategies DataFrame Shape: {myStrategies_df.shape}")


            AAPL-MAFlat  AMZN-MAFlat  EUR-MAShort  FBNDX-MAShort  GE-MAShort  \
2000-11-14          0.0          0.0    -0.000583       0.001428    0.025196   
2000-11-15         -0.0          0.0     0.000117       0.001426   -0.004751   
2000-11-16         -0.0         -0.0     0.005027       0.001423    0.002378   
2000-11-17         -0.0         -0.0     0.006349      -0.001423   -0.014354   
2000-11-20          0.0         -0.0    -0.004237       0.000000   -0.035565   

            GOLD-MAFlat  INTC-BB  SPY-MAFlat  
2000-11-14          0.0      0.0         0.0  
2000-11-15          0.0      0.0         0.0  
2000-11-16          0.0     -0.0        -0.0  
2000-11-17         -0.0      0.0        -0.0  
2000-11-20          0.0     -0.0        -0.0  
My Strategies DataFrame Shape: (5484, 8)


#### The start date of the "My Strategies" dataframe is same as that of Part A #8. (2000-11-14)

## Q4 Create the Max Sharpe Portfolio and calculate performance statistics for in-sample and out-sample data

In [13]:
# Storing weights calculated from Part A for Max Sharpe Portfolio.
weights = [0.194026, 0.078658, 0.157184, 0.211177, 0.090113, 0.123886, 0.079779, 0.065178]
# Calculate MAX-Sharpe Portfolio and add it to "My Strategies" portfolio.
max_sharpe_df = pd.DataFrame((weights * myStrategies_df).sum(axis=1, skipna=False), columns=['Max Sharpe Portfolio'])
myStrategies_df = myStrategies_df.join(max_sharpe_df, how='inner')

#### Calculating In-Sample data performance statistics and verifying with Part A

In [14]:
# Calculate Performance Statistics
calcPerformanceStatistics(myStrategies_df.loc[inSample])

Performance Statistics:


,AAPL-MAFlat,AMZN-MAFlat,EUR-MAShort,FBNDX-MAShort,GE-MAShort,GOLD-MAFlat,INTC-BB,SPY-MAFlat,Max Sharpe Portfolio
Average Annual Return,0.279595,0.219618,0.024583,0.015420,0.125548,0.063500,0.118114,0.057765,0.111012
Annual Risk,0.274548,0.331070,0.100898,0.038145,0.310752,0.144342,0.266639,0.106285,0.082109
Sharpe Ratio,1.018386,0.663359,0.243639,0.404243,0.404015,0.439929,0.442973,0.543493,1.352016
Max Drawdown,-0.560454,-0.547177,-0.287990,-0.126006,-0.906678,-0.395251,-0.503874,-0.194994,-0.124966
Success Ratio,0.659006,0.668729,0.515580,0.642431,0.518232,0.683978,0.738343,0.695691,0.548508


#### We can verify that the In-Sample data has statistics same as Part A data (in #16)

#### Calculating Out-Sample data performance statistics

In [15]:
# Calculate Performance Statistics
calcPerformanceStatistics(myStrategies_df.loc[outOfSample])

Performance Statistics:


,AAPL-MAFlat,AMZN-MAFlat,EUR-MAShort,FBNDX-MAShort,GE-MAShort,GOLD-MAFlat,INTC-BB,SPY-MAFlat,Max Sharpe Portfolio
Average Annual Return,0.248853,0.087724,0.043466,0.055264,-0.170227,0.072403,0.128410,0.081179,0.082852
Annual Risk,0.309429,0.243837,0.073029,0.053283,0.459837,0.136723,0.333044,0.146135,0.097081
Sharpe Ratio,0.804234,0.359767,0.595194,1.037173,-0.370189,0.529564,0.385563,0.555505,0.853435
Max Drawdown,-0.377275,-0.263749,-0.153604,-0.105729,-1.275920,-0.226672,-0.476805,-0.210084,-0.157603
Success Ratio,0.620438,0.690302,0.524505,0.641293,0.481752,0.678832,0.717414,0.676747,0.547445


#### The Sharpe for Out-Sample Max Sharpe Portfolio comes out to be 0.85, which is less than the In-Sample data.

## Q5 Comparing Out-Sample data with Equal Weight Portfolio

In [16]:
# Equal weights for the 8 instruments.
weights = np.ones(8)/8

# a. Benchmark-only portfolio (columns ending with -BMK-MA)
bmk_df = master_df.filter(like='-BMK-MA')
bmk_portfolio_returns = pd.DataFrame((weights * bmk_df).sum(axis=1, skipna=False), columns=['Benchmark Portfolio'])

In [17]:
# Calculate Performance Statistics
calcPerformanceStatistics(bmk_portfolio_returns.loc[outOfSample])

Performance Statistics:


,Benchmark Portfolio
Average Annual Return,0.063704
Annual Risk,0.180295
Sharpe Ratio,0.353333
Max Drawdown,-0.324084
Success Ratio,0.546403


#### The Sharpe ratio for the passive benchmark-only portfolio is 0.353

## Q6. Was it a waste of time?

#### (a) Performance of the active portfolio out-of-sample:

##### The active portfolio (Max Sharpe Portfolio) has shown solid performance in the out-of-sample period, delivering positive returns with a Sharpe ratio of 0.853. This indicates that the strategies applied to the portfolio were effective in managing risk and generating returns during the period from 2019-01-02 to 2022-10-31.

#### (b) Does the active portfolio outperform the passive portfolio?

##### Yes, the active portfolio outperformed the passive equally-weighted benchmark portfolio, which had a Sharpe ratio of 0.353. This difference in performance shows that the active portfolio delivered superior risk-adjusted returns compared to the passive benchmark.

### This shows that the project was not a waste of time, infact we were successfully able to grasp the concepts and deliver expected optimal results if not the best, 

"We pledge on our honor that we have not given nor received any unauthorized assistance on this
assignment to/from other groups. Nor have we received any assistance from prior year students."  
-- Abhishek Shetty, Akshay Belnekar, Rachita Fatehpuria, Vaibhavi Hedaoo